In [ ]:
import pandas as pd
import random
import os
import json
import math
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import brier_score_loss
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import itertools
import datetime as dt
start = dt.datetime.now()

In [ ]:
# Only execute this to run in a particular directory that you supply here.
os.chdir("/Users/dfox/WPI/WPIClasses/DS502StatisticalMethodsForDataScience/FinalProject/test")

In [ ]:
def print_time(x):
    print(f"{x}: elapsed time {dt.datetime.now() - start}")

In [ ]:
def do_hash(x):
    random.seed(23)
    return hash(x)

In [ ]:
CV_KFOLD = 5
START_TRAIN_YEAR = 1993
END_TEST_YEAR = 2023
CV_SPLIT_SIZE = math.floor((END_TEST_YEAR-START_TRAIN_YEAR) / (CV_KFOLD+2)) # 4
START_TEST_YEAR = START_TRAIN_YEAR + (CV_SPLIT_SIZE * (CV_KFOLD+1)) # 2017
BETTING_THRESHOLD = 6.0

In [ ]:
# Hyper parameters
# uncomment/comment to vary on the parameters you wish to
hyperparams = {'lag': {'V': [['NP', 'FirstDowns', 'RushAtt', 'RushYards', 
                              'RushTD',
                              'PassComp', 'PassAtt', 'PassYards', 'PassTD',
                              'PassInt', 'Sacked', 'SackedYards', 
                              'NetPassYards',
                              'NetTotalYards', 'Fumbles', 'FumblesLost',
                              'Turnovers', 'Penalties', 'PenaltyYards', 
                              'TimeOfPossesion', 'Q1S', 'Q2S', 'Q3S', 'Q4S',
                              'OTS']],
                       'N': [16, 28, 30, 41],
                       'S': [9, 5, 1]},
               # 'avg': {'N': [32, 24, 16, 8],
                       # 'W': [0.2, 0.1, 0.3],
                       # 'S': [9, 5, 1]}
               # 'avg': {'N': [36, 28],
               #         'W': [0.2],
               #         'S': [5]}
               # 'avg': {'N': [48, 40],
               #         'W': [0.2],
               #         'S': [5]}
               # 'avg': {'N': [44, 38],
               #         'W': [0.2],
               #         'S': [5]}
               # 'avg': {'N': [42, 39],
               #         'W': [0.2],
               #         'S': [5]}
               # 'avg': {'N': [41],
               #         'W': [0.2],
               #         'S': [5]}
                'avg': {'N': [28, 32, 36, 41],
                        'W': [0.2, 0.3],
                        'S': [5]}
               # 'avg': {'N': [41],
               #         'W': [0.15, 0.25],
               #         'S': [5]}
               # 'avg': {'N': [41],
               #         'W': [0.2],
               #         'S': [3, 4, 6, 7, 8]}

               }

In [ ]:
m_params = { 'linear':     {'model': LinearRegression,
                            'params': {},
                            'normalize': False
                            },
              'ridge':      {'model': Ridge,
                            'params': {'alpha': [0.1, 1.0, 10.0],
                                        'tol': [0.0001, 0.00001],
                                        'random_state': [23]
                                        },
                            'normalize': True
                            },
              'svr':        {'model': SVR,
                            'params': {'kernel': ['linear', 'poly', 'rbf'],
                                        'C': [0.1, 1.0],
                                        'tol': [0.0001, 0.00001],
                                        'degree': [2, 3]
                                        },
                            'normalize': True
                            },
              'rforest':    {'model': RandomForestRegressor,
                            'params': {'max_depth': [3, 5, 10],
                                        'n_estimators': [50, 100, 150],
                                        'random_state': [23],
                                        'n_jobs': [-1]
                                        },
                            'normalize': True
                            },
              'knn':        {'model': KNeighborsRegressor,
                            'params': {'n_neighbors': [1, 3, 5, 10, 15],
                                        'p': [1, 2]
                                        },
                            'normalize': True
                            }
           }

In [ ]:
def get_params_list(params_dict):
    keys = params_dict.keys()
    vals = params_dict.values()
    
    combinations = list(itertools.product(*vals))
    return [dict(zip(keys, combination)) for combination in combinations]

In [ ]:
def get_record_by_threshold(threshold_min, threshold_max, betspread,
                            predspread, gamespread):
    class_pred = predspread + betspread
    class_pred_list = class_pred.tolist()
    for i, pred in enumerate(class_pred_list):
        if abs(pred) < threshold_min or abs(pred > threshold_max):
            class_pred_list[i] = 0.0
    class_Y = gamespread + betspread
    class_Y_list = class_Y.tolist()
    wins = 0
    losses = 0
    for i in range(len(class_pred_list)):
        outcome = class_pred_list[i] * class_Y_list[i]
        if round(outcome, 2) > 0.00:
            wins += 1
        elif round(outcome, 2) < 0.00:
            losses += 1
    return wins, losses

In [ ]:
def X_construct_lag(hyper):
    N = hyper['N']
    # W = hyper['W']
    variables = hyper['V']
    print(variables)
    # X = hyper['X']
    
    x = '_'.join([f"{key}={str(val)}"
                  for key, val in sorted(hyper.items()) if key != 'V'])
    for var in variables:
        x+= f"_{var}"
    z = do_hash(x)

    if os.path.exists(f"NFLPred_lag_{z}.csv"):
        data = pd.read_csv(f"NFLPred_lag_{z}.csv")
        if "Unnamed: 0" in data:
            data.drop("Unnamed: 0", axis=1, inplace=True)
        print_time(f"Recieved X from cache file NFLPred_lag_{z}.csv")
        return data.copy()

    print_time(f"Construct lag X, will place in file NFLPred_lag_{z}.csv")

    games = pd.read_csv('NFLGamesDB19871993Fixed.csv', index_col=False)
    if "Unnamed: 0" in games:
        games.drop("Unnamed: 0", axis=1, inplace=True)
    games.sort_values(by = ['Year', 'Week', 'Date'], inplace = True,
                      ascending=True)
    # games.head()

    print_time("Loaded NFL Games")

    games['VNP'] = games.VScore - games.HScore
    games['HNP'] = games.HScore - games.VScore
    games['RegSeason'] = games.YearWeekDesc.str.contains('Week', na=True).astype(int)
    games['Playoffs'] = (games.RegSeason == 0).astype(int)
    games.drop('RegSeason', axis=1, inplace=True)
    games['Spread'] = games['VNP']
    games['BetSpread'] = games['VSpread']    
    
    names = pd.read_csv('TeamNamesMap.csv')

    print_time("Loaded NFL Teams Map, constructing symbol replacements")

    codes = pd.DataFrame(columns = ['Team_Name', 'Year', 'Team_Symbol'])
    for i in range(len(names)):
        name = names.iloc[i, 0]
        begin = names.iloc[i, 1]
        end = names.iloc[i, 2]
        symbol = names.iloc[i, 3]
        for y in range(begin, end + 1):
            patch = pd.DataFrame({'Team_Name': [name], 'Year': [y], 
                                  'Team_Symbol': [symbol]})
            codes = pd.concat([codes, patch], ignore_index = True, axis = 0)
        
    visitor = codes.copy().rename(columns = {'Team_Name': 'Visitor'})
    home = codes.copy().rename(columns = {'Team_Name': 'Home'})        
        
    games = pd.merge(games, visitor, on = ['Visitor', 'Year'])
    games = pd.merge(games, home, on = ['Home', 'Year'])
    games.sort_values(by = ['Year', 'Week', 'Date'], inplace = True)
        
    games.Visitor = games.Team_Symbol_x
    games.Home = games.Team_Symbol_y
    games.drop(['Team_Symbol_x', 'Team_Symbol_y'], axis = 1, inplace = True)
        
    game_data = pd.concat([games[['Year', 'Week', 'Date', 'Playoffs', 
                                  'Visitor', 'Home', 'IsNeutral', 'BetSpread', 
                                  'Spread']].copy(), 
                           games.copy().iloc[:, -61:-3]], axis = 1)
    game_data.drop(['HThirdDownConv', 'HThirdDownAtt', 'HFourthDownConv', 
                    'HFourthDownAtt', 'VThirdDownConv', 'VThirdDownAtt', 
                    'VFourthDownConv', 'VFourthDownAtt'], axis=1, inplace=True)
    game_data.dropna(inplace = True)
    print_time(f"cols={game_data.columns}")

    all_variables = ['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 
                     'IsNeutral', 'BetSpread', 'Spread'] + variables
    print_time(f"all variables{all_variables}")
    
    teams = list(game_data.Visitor.value_counts().index)
    game_list = ['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 
                 'IsNeutral', 'BetSpread', 'Spread']
    home_list = game_list + [col for col in game_data.columns
                             if col[0] == 'H' and col != 'Home']
    visit_list = game_list + [col for col in game_data.columns
                              if col[0] == 'V' and col != 'Visitor']
    team_stat = {}
    for team in teams:
        print_time(f"Constructing variables for team {team}")
        team_stat[team] = pd.DataFrame(columns = all_variables)
        temp = game_data[(game_data.Home == team)][home_list]
        temp.columns = all_variables
        team_stat[team] = pd.concat([team_stat[team], temp])
        temp = game_data[(game_data.Visitor == team)][visit_list]
        temp.columns = all_variables
        team_stat[team] = pd.concat([team_stat[team], temp])
        team_stat[team].sort_values(by = ['Year', 'Week', 'Date'], 
                                    inplace = True)
    
    lag_variables = []
    for i in range(1, N + 1):
        lag_variables += ['lag' + str(i) + col for col in game_data.columns 
                          if ((col[0] == 'H' and col != 'Home') or 
                              (col[0] == 'V' and col != 'Visitor' 
                               and col != 'VSpread'))]    
    
    past_game_stat = pd.DataFrame(columns = ['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 'IsNeutral', 'BetSpread', 'Spread'] + lag_variables)
    past_game_stat[['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home',
                    'IsNeutral', 'BetSpread', 'Spread']] =\
        game_data[['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home',
                   'IsNeutral', 'BetSpread', 'Spread']]
    
    past_game_stat = pd.DataFrame(columns = ['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 'IsNeutral', 'BetSpread', 'Spread'] + lag_variables)
    past_game_stat[['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 
                    'IsNeutral', 'BetSpread', 'Spread']] =\
        game_data[['Year', 'Week', 'Date', 'Playoffs', 'Visitor', 'Home', 
                   'IsNeutral', 'BetSpread', 'Spread']]
    past_game_stat.sort_values(by = ['Year', 'Week', 'Date'], inplace = True)
    for team in teams:
        print_time(f"Constructing lag variables for team {team}")
        home_dates = past_game_stat.loc[past_game_stat['Home'] == team, 'Date']
        visit_dates = past_game_stat.loc[past_game_stat['Visitor'] == team, 
                                         'Date']
        temp = team_stat[team].copy()
        for i in range(1, N + 1):
            lag_values = temp.iloc[:, 9:].shift(i)
            home_vars = [col for col in lag_variables 
                         if col.startswith('lag' + str(i) + 'H')]
            visit_vars = [col for col in lag_variables 
                          if col.startswith('lag' + str(i) + 'V')]
            past_game_stat.loc[game_data.Home == team, home_vars] =\
                lag_values[temp.Date.isin(home_dates)].values
            past_game_stat.loc[game_data.Visitor == team, visit_vars] =\
                lag_values[temp.Date.isin(visit_dates)].values    

    past_game_stat.sort_values(by = ['Year', 'Week', 'Date'], inplace = True)
    past_game_stat.dropna(inplace = True)
    
    
    data = past_game_stat.copy().merge(games[['Year', 'Week', 'Date', 
                                              'Playoffs', 'Visitor', 'Home', 
                                              'IsNeutral', 'BetSpread', 
                                              'Spread']],
                                       on = ['Year', 'Week', 'Date', 
                                             'Playoffs', 'Visitor', 'Home', 
                                             'IsNeutral', 'BetSpread', 
                                             'Spread'])

    data.to_csv(f"NFLPred_lag_{z}.csv")
    
    return data.copy()

In [ ]:
def X_construct_avg(hyper):
    N = hyper['N']
    W = hyper['W']
    if 'V' in hyper and (len(hyper['V']) > 1 or hyper['V'][0] != 'NP'):
        return None
    variables = ['NP']
    
    x = '_'.join([f"{key}={str(val)}"
                  for key, val in sorted(hyper.items()) if key != 'V'])
    for var in variables:
        x += f"_{var}"
    z = do_hash(x)

    if os.path.exists(f"NFLPred_avg_{z}.csv"):
        gdl_df = pd.read_csv(f"NFLPred_avg_{z}.csv")
        if "Unnamed: 0" in gdl_df:
            gdl_df.drop("Unnamed: 0", axis=1, inplace=True)
        print_time(f"Recieved X from cache file NFLPred_avg_{z}.csv")
        return gdl_df.copy()

    print_time(f"Construct avg X, will place in file NFLPred_avg_{z}.csv")

    games = pd.read_csv('NFLGamesDB19871993Fixed.csv', index_col=False)
    if "Unnamed: 0" in games:
        games.drop("Unnamed: 0", axis=1, inplace=True)
    games.sort_values(by = ['Year', 'Week', 'Date'], inplace = True, 
                      ascending=True)

    names = pd.read_csv('TeamNamesMap.csv')

    codes = pd.DataFrame(columns = ['Team_Name', 'Year', 'Team_Symbol'])
    for i in range(len(names)):
        name = names.iloc[i, 0]
        begin = names.iloc[i, 1]
        end = names.iloc[i, 2]
        symbol = names.iloc[i, 3]
        for y in range(begin, end + 1):
            patch = pd.DataFrame({'Team_Name': [name], 'Year': [y], 'Team_Symbol': [symbol]})
            codes = pd.concat([codes, patch], ignore_index = True, axis = 0)
        
    visitor = codes.copy().rename(columns = {'Team_Name': 'Visitor'})
    home = codes.copy().rename(columns = {'Team_Name': 'Home'})        
        
    games = pd.merge(games, visitor, on = ['Visitor', 'Year'])
    games = pd.merge(games, home, on = ['Home', 'Year'])
    games.sort_values(by = ['Year', 'Week', 'Date'], inplace = True)
        
    games.Visitor = games.Team_Symbol_x
    games.Home = games.Team_Symbol_y
    games.drop(['Team_Symbol_x', 'Team_Symbol_y'], axis = 1, inplace = True)
        
    games['Spread'] = games['VScore'] - games['HScore']
    games['RegSeason'] = games.YearWeekDesc.str.contains('Week', na=True).\
        astype(int)
    games['Playoffs'] = (games.RegSeason == 0).astype(int)
    games['BetSpread'] = games['VSpread']
    games.drop(['RegSeason', 'VSpread'], axis=1, inplace=True)
        
    teams = list(games.Visitor.value_counts().index)
    calculated_variables = ['VNRPA', 'VNYA'] + teams
    inherited_variables = ['Year', 'Week', 'Date', 'Visitor', 'Home', 
                           'IsNeutral', 'Spread', 'BetSpread', 'Playoffs']

    gdl_df = games[games.Year >= 1973].copy()
    gdl_df = gdl_df[inherited_variables].copy()
    for var in calculated_variables:
        gdl_df[var] = 0.0

    i = 0
    for index, row in gdl_df.iterrows():
        if i % 100 == 0:
            print_time(f"processing row {i} for gdl_df")
        i += 1
        v_team = row.Visitor
        v_games = games[((games.Visitor == v_team) | (games.Home == v_team))]
        v_games = v_games[((v_games.Year < row.Year)
                           | ((v_games.Year == row.Year) 
                              & (v_games.Week < row.Week)))]
        v_games = v_games[-N:]
        h_team = row.Home
        h_games = games[((games.Visitor == h_team) 
                         | (games.Home == h_team))]
        h_games = h_games[((h_games.Year < row.Year) 
                           | ((h_games.Year == row.Year) 
                              & (h_games.Week < row.Week)))]
        h_games = h_games[-N:]
        visitor_net_reg_points_total = 0.0
        visitor_net_yards_total = 0.0
        divisor = 0.0
        teams_dict = {team: 0.0 for team in teams}
        for v_index, v_row in v_games.iterrows():
            weight = W**(row.Year - v_row.Year)
            assert(weight > 0.0)
            visitor_net_reg_points = weight*((v_row['VScore'] - v_row['VOTS'])
                                             - (v_row['HScore'] 
                                                - v_row['HOTS']))
            visitor_net_yards = weight*(v_row['VNetTotalYards']
                                        - v_row['HNetTotalYards'])
            divisor += weight
            if v_row.Visitor == v_team:
                visitor_net_reg_points_total += visitor_net_reg_points
                visitor_net_yards_total += visitor_net_yards
                teams_dict[v_row.Home] += weight
            else:
                visitor_net_reg_points_total -= visitor_net_reg_points
                visitor_net_yards_total -= visitor_net_yards
                teams_dict[v_row.Visitor] += weight
        if divisor == 0.0:
            # no previous games played, expansion team, just set averages to 0
            visitor_net_reg_points_avg = 0.0
            visitor_net_yards_avg = 0.0
        else:
            visitor_net_reg_points_avg = visitor_net_reg_points_total / divisor
            visitor_net_yards_avg = visitor_net_yards_total / divisor
        visitor_net_reg_points_total = 0.0
        visitor_net_yards_total = 0.0
        divisor = 0.0
        for h_index, h_row in h_games.iterrows():
            weight = W**(row.Year - h_row.Year)
            assert(weight > 0.0)
            visitor_net_reg_points = weight*((h_row['VScore'] - h_row['VOTS']) 
                                             - (h_row['HScore'] 
                                                - h_row['HOTS']))
            visitor_net_yards = weight*(h_row['VNetTotalYards'] 
                                        - h_row['HNetTotalYards'])
            divisor += weight
            if h_row.Visitor == h_team:
                visitor_net_reg_points_total += visitor_net_reg_points
                visitor_net_yards_total += visitor_net_yards
                teams_dict[h_row.Home] -= weight
            else:
                visitor_net_reg_points_total -= visitor_net_reg_points
                visitor_net_yards_total -= visitor_net_yards
                teams_dict[h_row.Visitor] -= weight
        if divisor == 0.0:
            # no previous games played, expansion team, just set averages to 0
            home_net_reg_points_avg = 0.0
            home_net_yards_avg = 0.0
        else:
            home_net_reg_points_avg = visitor_net_reg_points_total / divisor
            home_net_yards_avg = visitor_net_yards_total / divisor
        gdl_df.at[index, 'VNRPA'] = (visitor_net_reg_points_avg 
                                     - home_net_reg_points_avg)
        gdl_df.at[index, 'VNYA'] = (visitor_net_yards_avg - home_net_yards_avg)
        for team in teams_dict:
            gdl_df.at[index, team] = teams_dict[team]
            
    gdl_df.drop('VNYA', axis=1, inplace=True)
    
    gdl_df.to_csv(f"NFLPred_avg_{z}.csv")
    
    return gdl_df.copy()

In [ ]:
def normalize_X(df, cols):
    cdf = df.copy()
    for col in cols:
        if df[col].max() == cdf[col].min():
            if cdf[col].max() < 0.5:
                cdf[col] = 0
            else:
                cdf[col] = 1
        else:
            cdf[col] = (cdf[col] - cdf[col].min())/(cdf[col].max() 
                                                    - cdf[col].min())
    return cdf   

In [ ]:
def hash_cv_results_input(cv_results_input):
    xstr = json.dumps(cv_results_input)
    return str(do_hash(xstr))

def cv_results_insert(all_dict, cv_results):
    cv_results_input_hash = hash_cv_results_input(cv_results['input'])
    if cv_results_input_hash in all_dict:
        for i, r in enumerate(all_dict[cv_results_input_hash]):
            if r['input'] == cv_results['input']:
                all_dict[cv_results_input_hash][i] = cv_results
                return
        all_dict[cv_results_input_hash].append(cv_results)
        return
    all_dict[cv_results_input_hash] = [cv_results]
    with open("cv_results_nfl.json", "w") as fp:
        json.dump(all_dict, fp, indent=4)
    
def cv_results_get(all_dict, cv_results_input):
    cv_results_input_hash = hash_cv_results_input(cv_results_input)
    if cv_results_input_hash in all_dict:
        for r in all_dict[cv_results_input_hash]:
            if r['input'] == cv_results_input:
                return r
    return None

# cv_results_get_best(all_dict, mae, False, ['rforest'], ['lag']
                    
def cv_results_get_best(all_dict, tag, maximize, method_filter=None,type_filter=None):
    all_list = []
    
    for cv_results in all_dict:
        for result in all_dict[cv_results]:
            if (method_filter is not None
                and result['input']['method'] in method_filter):
                           
                if (type_filter is not None
                    and result['input']['Xtype'] in type_filter):
                    all_list.append(result)
                elif type_filter is None:
                    all_list.append(result)
            elif method_filter is None:
                all_list.append(result)
    if len(all_list) == 0:
        return None
    all_list_sorted = sorted(all_list, key=lambda x: x['cv_output'][tag],
                             reverse=maximize)
    return all_list_sorted[0]

In [ ]:
# Cross-validation
# Only run if you wish to tune parameters for the five models in m_params above
# Also tunes hyper params in hyperparams above

# Saves all cross-validation tuning runs to cv_results_nfl.json in the local directory
# Loads that file to begin with so that duplicate runs are not executed
# Saves every single cross-validation run for a given parameter combination

# If you wish to generate new runs of cross-validation tuning runs, make sure you have
# deleted/renamed the cv_results_nfl.json file

# Must execute all above before running this

if os.path.exists("cv_results_nfl.json"):
    with open("cv_results_nfl.json", "r") as fp:
        all_cv_results = json.load(fp)
else:
    all_cv_results = {}

# Cross-validation to find best parameters
hyperparams_lag = get_params_list(hyperparams['lag'])
hyperparams_avg = get_params_list(hyperparams['avg'])

threshold_list = []

for LAG in [True, False]:
    if LAG:
        hyperparams = hyperparams_lag
    else:
        hyperparams = hyperparams_avg

    for hyper in hyperparams:
        if LAG:
            X = X_construct_lag(hyper)
        else:
            X = X_construct_avg(hyper)
        if X is None:
            continue
        
        for method in m_params:
            print_time(f"{method}")
            hash_method = do_hash(method)
            if m_params[method]['normalize']:
                data = normalize_X(X, [col for col in X][9:])
            else:
                data = X.copy()
            params_list = get_params_list(m_params[method]['params'])
            if len(params_list) > 0:
                params_list.append({})
            random.shuffle(params_list)
            
            for params in params_list:
                
                cv_results_input = {}
                if LAG:
                    cv_results_input['Xtype'] = 'lag'
                else:
                    cv_results_input['Xtype'] = 'avg'
                cv_results_input['hyper'] = hyper
                cv_results_input['method'] = method
                cv_results_input['params'] = params
                cv_results_input['start_train_year'] = START_TRAIN_YEAR
                cv_results_input['start_test_year'] = START_TEST_YEAR
                cv_results_input['kfold'] = CV_KFOLD
                cv_results_input['split_size'] = CV_SPLIT_SIZE
                cv_results = cv_results_get(all_cv_results, cv_results_input)
                if cv_results is not None:
                    print("already calculated...")
                    continue #already calculated
                
                rmse_accum = 0.0
                mae_accum = 0.0
                dollars_accum = 0
                accuracy_accum = 0.00
                # threshold_accum = 0
                r2_accum = 0.0
                max_dollars = None
                best_threshold = None
                for i in range(CV_KFOLD):
                    year = START_TRAIN_YEAR + i*CV_SPLIT_SIZE
                    train_data = X[((X.Year >= START_TRAIN_YEAR) 
                                    & (X.Year < year+CV_SPLIT_SIZE)
                                    & (X.Week >= hyper['S']))].copy()
                    valid_data = X[((X.Year >= year+CV_SPLIT_SIZE) 
                                    & (X.Year < year+(2*CV_SPLIT_SIZE))
                                    & (X.Week >= hyper['S']))]
                
                    variables = [col for col in X.columns 
                                 if col not in ['Year', 'Week', 'Date', 
                                                'Visitor', 'Home', 'Spread', 
                                                'BetSpread']]
                    train_X = train_data[variables].copy()
                    valid_X = valid_data[variables].copy()
                    train_Y = train_data['Spread']
                    valid_Y = valid_data['Spread']
                    valid_spread_Y = valid_data['BetSpread']
                    valid_class_Y = 0
                    if len(params) > 0:
                        model = m_params[method]['model'](**params)\
                            .fit(train_X, train_Y)
                    else:
                        model = m_params[method]['model']()\
                            .fit(train_X, train_Y)
                    valid_pred_Y = model.predict(valid_X)
                    rmse = mean_squared_error(valid_Y, valid_pred_Y, 
                                              squared=False)
                    rmse_accum += rmse
                    mae = mean_absolute_error(valid_Y, valid_pred_Y) 
                    mae_accum += mae
                    r2 = model.score(valid_X, valid_Y)
                    wins, losses = get_record_by_threshold(BETTING_THRESHOLD,
                                                           50,
                                                           valid_spread_Y, 
                                                           valid_pred_Y,
                                                           valid_Y)
                    accuracy = wins / (wins + losses)
                    accuracy_accum += accuracy
                    r2_accum += r2
                rmse = rmse_accum / CV_KFOLD
                mae = mae_accum / CV_KFOLD
                accuracy = accuracy_accum / CV_KFOLD
                r2 = r2_accum / CV_KFOLD
                
                print_time(f"{method} {hyper} {params} {rmse} {mae} {accuracy}")
                cv_results = {}
                cv_results['input'] = cv_results_input
                cv_results['cv_output'] = {}
                cv_results['cv_output']['rmse'] = rmse
                cv_results['cv_output']['mae'] = mae
                cv_results['cv_output']['r2'] = r2
                cv_results['cv_output']['accuracy'] = accuracy
                cv_results_insert(all_cv_results, cv_results)

In [ ]:
def prob_by_ps(ps):
    return 1 / ((10**(-ps/16)) + 1)

def win_by_ps(ps):
    return ps > 0

In [ ]:
def calc_vegas_brier(df):
    return brier_score_loss((df['Spread'] > 0), prob_by_ps(-df['BetSpread']))

def calc_vegas_rmse(df):
    return mean_squared_error(df['Spread'], -df['BetSpread'], squared=False)

def calc_our_brier(df, variables, model):
    test_prob_pred_Y = prob_by_ps(model.predict(df[variables]))
    return brier_score_loss((df['Spread'] > 0), test_prob_pred_Y)
    
def calc_our_r2(df, variables, model):
    return model.score(df[variables], df['Spread'])
    
def calc_our_rmse(df, test_pred_Y):
    return mean_squared_error(df['Spread'], test_pred_Y, squared=False)
    
def calc_elo_brier():
    df = pd.read_csv("nfl_elo2.csv")
    df = df[df.season >= 2017].copy()
    df['Spread'] = df.score2 - df.score1
    df_no_ties = df[df['Spread'] != 0]
    return brier_score_loss((df_no_ties['Spread'] > 0), df_no_ties.elo_prob2)
    
def calc_elo_rmse():
    df = pd.read_csv("nfl_elo2.csv")
    df = df[df.season >= 2017].copy()
    df['Spread'] = df.score2 - df.score1
    df_no_ties = df[df.Spread != 0]
    return mean_squared_error(df_no_ties.Spread, df_no_ties.spread1_pre, 
                              squared=False)
    
def calc_vegas_mae(df):
    return mean_absolute_error(df['Spread'], -df['BetSpread'])
     
def calc_our_mae(df, test_pred_Y):
    return mean_absolute_error(df['Spread'], test_pred_Y)
    
def calc_elo_mae():
    df = pd.read_csv("nfl_elo2.csv")
    df = df[df.season >= 2017].copy()
    df['Spread'] = df.score2 - df.score1
    df_no_ties = df[df.Spread != 0]
    return mean_absolute_error(df_no_ties.Spread, df_no_ties.spread1_pre)

In [ ]:
# Results generation
# Generates results against the test data set
# Reads cv_results_nfl.json and sorts based on the desired metric
# Selects top cross-validation parameters for each method

# Places the results in test_results_nfl.json, which is a dictionary of metrics as keys
# and a list of results, one for each method

# Must execute everything above
# However, for the cross-validation cell, you do not need to run that if
# you have already done so

# If you do execute the cross-validation cell, but there is a 
# cv_results_nfl.json, it should be harmless because if 
# a hyper parameter / method parameter / method combination has been
# run before (appears in cv_results_nfl.json), it will simply skip over that
# combination and not fit the model again

start = dt.datetime.now()

print_time("start")
if os.path.exists("cv_results_nfl.json"):
    with open("cv_results_nfl.json", "r") as fp:
        all_cv_results = json.load(fp)
else:
    exit(-1)

metrics = ['mae']
Xtypes = ['avg', 'lag']
all_test_results = {}

save_fit = None
save_test_Y = None
save_test_X = None
save_test_pred_Y = None

for metric in metrics:
    print_time(f"metric {metric}")
    for Xtype in Xtypes:
        print_time(f"X type {Xtype}")
        for method in m_params:
            print_time(f"method {method} in params {m_params[method]['params']}")
            if metric not in all_test_results:
                all_test_results[metric] = {}
            if Xtype not in all_test_results[metric]:
                all_test_results[metric][Xtype] = []
            results = cv_results_get_best(all_cv_results, metric, 
                                          metric=='accuracy',
                                          [method], [Xtype])
            if results is None:
                continue
            print_time(f"Optimizing {metric}, results {results}")
            hyper = results['input']['hyper']
            
            if results['input']['Xtype'] == 'lag':
                X = X_construct_lag(hyper)
            else:
                X = X_construct_avg(hyper)
                
            print(f"X constructed {X}")
            
            if m_params[results['input']['method']]['normalize']:
                data = normalize_X(X, [col for col in X][9:])
            else:
                data = X.copy()
            
            
            variables = [col for col in data.columns 
                         if col not in ['Year', 'Week', 'Date', 'Visitor', 
                                        'Home', 'Spread', 'BetSpread']]
            train_data = data\
                [((data.Year >= results['input']['start_train_year']) 
                  & (data.Year < results['input']['start_test_year'])
                  & (data.Week >= hyper['S']))]
            test_data = data\
                [((data.Year >= results['input']['start_test_year']) 
                  & (data.Year <= END_TEST_YEAR) 
                  & (data.Week >= hyper['S']))]
            print_time(test_data[variables])
            train_Y = train_data['Spread']
            test_Y = test_data['Spread']
            test_betspread_Y = test_data['BetSpread']
              
            model = m_params[results['input']['method']]['model']
            params = results['input']['params']
            
            print_time(f"about to fit model {model}")
            
            if len(params) > 0:
                fit = model(**params).fit(train_data[variables],
                                          train_data['Spread'])
            else:
                fit = model().fit(train_data[variables], train_data['Spread'])
            print_time("model was fit")    
            test_pred_Y = fit.predict(test_data[variables])
            if method == 'linear' and Xtype == 'avg':
                save_fit = fit
                save_test_Y = test_Y
                save_test_X = test_data[variables]
                save_test_pred_Y = test_pred_Y
            
            test_prob_pred_Y = prob_by_ps(test_pred_Y)
            wins, losses = get_record_by_threshold(6.0, 50.0, 
                                                   #BETTING_THRESHOLD,
                                                   # 50,
                                                   # 5.0,
                                                   #calc_our_mae(test_data, 
                                                   #             test_pred_Y),
                                                   test_betspread_Y, 
                                                   test_pred_Y,
                                                   test_data.Spread)
            
        
            dollars = wins * 100 - losses * 115
            accuracy = wins / (wins + losses)
            
            results['test_output'] = {}
            to = results['test_output']
            # to['threshold'] = threshold
            to['accuracy'] = accuracy
            to['wins'] = wins
            to['losses'] = losses
            to['dollars'] = dollars
            print_time("")
            print("")
            print(results)
            print("With test:")
            print(f"Best lowest week {hyper['S']}")
            # print(f"best threshold {threshold}")
            print(f"best method {results['input']['method']}")
            print(f"best params {results['input']['params']}")
            print(f"best accuracy {accuracy}")
            print(f"best wins {wins}")
            print(f"best losses {losses}")
            print(f"best dollars {dollars}")
            
            to['our_rmse'] = calc_our_rmse(test_data, test_pred_Y)
            to['vegas_rmse'] = calc_vegas_rmse(test_data)
            to['elo_rmse'] = calc_elo_rmse()
            
            print(f"Our root mean squared error is {to['our_rmse']}")
            print(f"Vegas root mean squared error is {to['vegas_rmse']}")
            print(f"ELO root mean squared error is {to['elo_rmse']}")
            
            to['our_mae'] = calc_our_mae(test_data, test_pred_Y)
            to['vegas_mae'] = calc_vegas_mae(test_data)
            to['elo_mae'] = calc_elo_mae()
            
            print(f"Our mean absolute error is {to['our_mae']}")
            print(f"Vegas mean absolute error is {to['vegas_mae']}")
            print(f"ELO mean absolute error is {to['elo_mae']}")
            
            test_data_no_ties = test_data[test_data['Spread'] != 0]
            to['our_brier'] = calc_our_brier(test_data_no_ties, variables, fit)
            to['vegas_brier'] = calc_vegas_brier(test_data_no_ties)
            to['elo_brier'] = calc_elo_brier()
            
            print(f"Our Brier Score is {to['our_brier']}")
            print(f"Vegas Brier Score is {to['vegas_brier']}")
            print(f"ELO Brier Score is {to['elo_brier']}")
            
            to['our_r2'] = calc_our_r2(test_data, variables, fit)
        
            print(f"Our R-squared is {to['our_r2']}")
            
            all_test_results[metric][Xtype].append(results)
            if method == 'linear':
                t = zip(fit.feature_names_in_, fit.coef_)
                print(list(t))
                print(fit.intercept_)
        
with open("test_results_nfl.json", "w") as fp:
    json.dump(all_test_results, fp, indent=4)

In [ ]:
import matplotlib.pyplot as plt
resid = save_test_Y - save_test_pred_Y
plt.plot(save_test_X['VNRPA'], resid, 'o')
plt.xlabel("VNRPA")
plt.ylabel("Residuals")
plt.title("Residual plot of best fit: Linear Regression")
plt.show()

In [ ]:
features_list = [("intercept", save_fit.intercept_)]
features_list.extend(list(zip(save_fit.feature_names_in_, save_fit.coef_)))
team_list = features_list[5:]
team_sorted_list = sorted(team_list, key=lambda x: x[1])
features_list = features_list[:4]
features_list.extend(team_sorted_list)
df = pd.DataFrame(features_list, columns=['Feature', 'Coefficient'])
df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(save_test_X['VNRPA'], save_test_Y, 'o')

#obtain m (slope) and b(intercept) of linear regression line
m2, m1, b = np.polyfit(save_test_X['VNRPA'], save_test_Y, 2)

#add linear regression line to scatterplot 
plt.plot(save_test_X['VNRPA'], m2*(save_test_X['VNRPA']**2) + m1*save_test_X['VNRPA'] +b)
plt.xlabel('VNRPA')
plt.ylabel("True point spread")
plt.title("Plot of VNRPA vs. true point spread, with fitted polynomial")
plt.show()
